In [ ]:
import os, sys, warnings
import scanpy as sc
import squidpy as sq
import numpy as np
import pandas as pd

import anndata as ad
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy import sparse
from matplotlib.ticker import PercentFormatter

from abc_atlas_access.abc_atlas_cache.abc_project_cache import AbcProjectCache

from pathlib import Path

sc.set_figure_params(dpi=120, facecolor="white")
sc.settings.verbosity = 2

# dossier pour sauvegarder les figures
sc.settings.figdir = "fig"
os.makedirs(sc.settings.figdir, exist_ok=True)

In [ ]:
# Loader ABCA MERFISH

# Pour les warnings
warnings.filterwarnings("ignore", category=FutureWarning)
try:
    import dask
    dask.config.set({"dataframe.query-planning": True})
except Exception:
    pass

from abc_atlas_access.abc_atlas_cache.abc_project_cache import AbcProjectCache

#Cache
cache_dir = Path("./data/abc_atlas")
cache_dir.mkdir(parents=True, exist_ok=True)
abc = AbcProjectCache.from_cache_dir(cache_dir)
try:
    abc.load_latest_manifest()
except Exception as e:
    print("Note: impossible de charger le manifeste le plus récent (ok) :", e)

def _harmonize_gene_names(adata, to_upper=True):
    """Harmonise les noms de gènes pour matcher tes scripts (MAJUSCULES par défaut)."""
    if to_upper:
        adata.var_names = [g.upper() for g in adata.var_names]
    else:
        adata.var_names = [g[:1].upper()+g[1:].lower() if g else g for g in adata.var_names]
    adata.var_names_make_unique()

def _align_cell_meta_index(adata, cell_meta):
    """Essaie d'aligner l'index des métadonnées sur les obs_names de adata."""
    # Si l'index actuel matche déjà bien, on garde
    if cell_meta.index.isin(adata.obs_names).mean() > 0.9:
        return cell_meta.reindex(adata.obs_names)

    # Essais de colonnes candidates d'ID cellule
    candidates = [
        "cell_label", "cell_id", "barcode", "obs_name", "obs_names", "cell", "index"
    ]
    for c in candidates:
        if c in cell_meta.columns:
            cm = cell_meta.set_index(c)
            if cm.index.isin(adata.obs_names).mean() > 0.9:
                return cm.reindex(adata.obs_names)

    # Dernier recours: si une colonne unique a même longueur, on tente
    for c in cell_meta.columns:
        if cell_meta[c].is_unique and cell_meta.shape[0] >= adata.n_obs:
            cm = cell_meta.set_index(c)
            if cm.index.isin(adata.obs_names).mean() > 0.9:
                return cm.reindex(adata.obs_names)

    # Sinon: on reindexe à vide (mieux que planter)
    return cell_meta.reindex(adata.obs_names)

def _guess_xy(df):
    """Devine des colonnes x/y plausibles dans un DataFrame de métadonnées."""
    cands_x = [c for c in df.columns if c.lower() in ("x","x_um","x_ccf","x_centroid","x_spatial")]
    cands_y = [c for c in df.columns if c.lower() in ("y","y_um","y_ccf","y_centroid","y_spatial")]
    if not cands_x or not cands_y:
        cands_x = [c for c in df.columns if c.lower().startswith("x")]
        cands_y = [c for c in df.columns if c.lower().startswith("y")]
    if not cands_x or not cands_y:
        raise ValueError("Impossible de deviner les colonnes x/y dans obs; vérifie adata.obs.columns.")
    return cands_x[0], cands_y[0]

#def loader
def load_abca(animal="Zhuang-ABCA-1", mode="log2", use_backed=False, harmonize_genes=True):
    """
    Charge un dataset MERFISH ABCA pour un usage "compatible" avec tes scripts existants.
    - animal: 'Zhuang-ABCA-1' .. 'Zhuang-ABCA-4'
    - mode:   'log2' (normalisé/transformé) ou 'raw' (comptages)
    - use_backed: lecture mappée, utile pour inspection sans tout charger en RAM
    - harmonize_genes: passe les gènes en MAJUSCULES (True par défaut)
    Retourne: AnnData avec .obsm['spatial'] rempli et obs enrichi.
    """
    assert mode in ("log2", "raw"), "mode doit être 'log2' ou 'raw'"

    # a) Expression
    file_name = f"{animal}/{mode}"  # nom logique côté ABCA
    fpath = abc.get_data_path(directory=animal, file_name=file_name)
    adata = ad.read_h5ad(fpath, backed="r" if use_backed else None)

    # b) Métadonnées cellules
    cell_meta = abc.get_metadata_dataframe(directory=animal, file_name="cell_metadata")

    # c) Alignement des index
    cell_meta = _align_cell_meta_index(adata, cell_meta)

    # d) Join safe: ajoute que les colonnes qui existent pas déjà
    overlap = adata.obs.columns.intersection(cell_meta.columns)
    if len(overlap) > 0:
        print(f"[info] Colonnes communes non dupliquées: {len(overlap)} (préservées côté adata.obs)")
    to_add = cell_meta.columns.difference(adata.obs.columns)
    adata.obs = adata.obs.join(cell_meta[to_add], how="left")

    # e) Coordonnées spatiales 2D attendues par Squidpy/Scanpy
    try:
        xcol, ycol = _guess_xy(adata.obs)
        adata.obsm["spatial"] = adata.obs[[xcol, ycol]].to_numpy()
        print(f"[spatial] Colonnes utilisées: {xcol}, {ycol}")
    except Exception as e:
        print("[avertissement] Impossible d'établir obsm['spatial'] automatiquement:", e)

    # f) Harmonise les noms de gènes
    if not use_backed and harmonize_genes:
        _harmonize_gene_names(adata, to_upper=True)

    # g) Clé 'spatial' minimale
    adata.uns.setdefault("spatial", {})

    return adata

In [ ]:
# 3)Chargement du dataset
ANIMAL = "Zhuang-ABCA-1"
MODE   = "raw"
BACKED = False       
adata = load_abca(ANIMAL, mode=MODE, use_backed=BACKED, harmonize_genes=True)
print(adata)  # n_obs x n_vars

##Remplacer les var_names (Ensembl) par les symboles de gène
# 1) Sauvegarder les Ensembl dans une colonne
adata.var["ensembl_id"] = adata.var_names

# 2) Récupérer les symboles
if "gene_symbol" not in adata.var.columns:
    raise ValueError("Ce dataset ne contient pas la colonne 'gene_symbol' dans .var")

symbols = adata.var["gene_symbol"].astype(str)

# 3) Option: nettoyer/normaliser les symboles
#    a) version officielle (tout majuscule)
symbols_official = symbols.str.upper()

#    b) version jolie (1re lettre majuscule)
symbols_pretty = symbols.str.capitalize()

# 4) Choisir préférence
USE_PRETTY = True  # =jolie

new_names = symbols_pretty if USE_PRETTY else symbols_official

# 5) Symboles peuvent être manquants : on garde l'Ensembl
mask_bad = new_names.isna() | (new_names.str.len() == 0)
new_names = new_names.where(~mask_bad, adata.var["ensembl_id"])

# 6) Appliquer et garantir l’unicité
adata.var_names = new_names.values
adata.var_names_make_unique()

print("Exemples de gènes:", list(adata.var_names[:12]))

In [ ]:
# Premier filtre : on enlève les celulles sans métadonnées

n_total = adata.n_obs
n_missing = adata.obs['abc_sample_id'].isna().sum()

print(f"Total cellules : {n_total:,}")
print(f"Cellules sans métadonnées : {n_missing:,} ({n_missing/n_total*100:.1f}%)")

adata = adata[~adata.obs['abc_sample_id'].isna()].copy()

print(f"Après filtrage : {adata.n_obs:,} cellules restantes ({adata.n_obs/n_total*100:.1f}%)")

In [ ]:
# 0) Réduire la taille (moins lourd)
adata_ref = adata.copy()
sc.pp.subsample(adata_ref, fraction=0.1, random_state=0)
print("Taille référence:", adata_ref.shape)

In [ ]:
# Calcul indicateurs qualité

if "counts" in adata_ref.layers:
    sc.pp.calculate_qc_metrics(adata_ref, layer="counts", percent_top=[50], inplace=True)
else:
    sc.pp.calculate_qc_metrics(adata_ref, percent_top=[50], inplace=True)


# 1) Filtrage cellules

qc = adata_ref.obs.copy()

def iqr_bounds(s, k=3):
    s = s.dropna()
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    return q1 - k*iqr, q3 + k*iqr

low_counts, high_counts = iqr_bounds(qc['total_counts'])

if 'pct_counts_in_top_50_genes' in qc.columns:
    _, high_pt = iqr_bounds(qc['pct_counts_in_top_50_genes'])
    max_pt = float(min(80, high_pt))
else:
    max_pt = None

# seuil min forcé à 50
keep = (
    (qc['n_genes_by_counts'] >= 50) &
    (qc['total_counts'] <= min(high_counts, qc['total_counts'].quantile(0.99)))
)
if max_pt is not None:
    keep &= (qc['pct_counts_in_top_50_genes'] <= max_pt)

adata_ref = adata_ref[keep].copy()
print("Après QC (cellules):", adata_ref.n_obs)





# 2) Filtrage gènes (bruit, gènes trop rares)
# Enlève les gènes vus dans très peu de cellules.

# gènes exprimés dans au moins X cellules
min_cells = max(5, int(0.005 * adata_ref.n_obs))  # 0.5% des cellules, plancher=5
sc.pp.filter_genes(adata_ref, min_cells=min_cells)
print("Après QC (gènes):", adata_ref.n_vars)

In [ ]:
qc = adata_ref.obs
print("Min avant filtrage :", qc['n_genes_by_counts'].min())

In [ ]:
if "counts" in adata.layers:
    sc.pp.calculate_qc_metrics(adata, layer="counts", percent_top=[50], inplace=True)
else:
    sc.pp.calculate_qc_metrics(adata, percent_top=[50], inplace=True)

In [ ]:
sc.pl.violin(
    adata,
    keys=['n_genes_by_counts', 'total_counts', 'pct_counts_in_top_50_genes'],
    jitter=0,              
    stripplot=False,       
    multi_panel=True,      
)


sc.pl.violin(
    adata_ref,
    ['n_genes_by_counts', 'total_counts', 'pct_counts_in_top_50_genes'],
    jitter=0,
    stripplot=False,
    multi_panel=True
)

In [ ]:
# 2) Normalisation + log 

adata_raw = adata_ref.copy()  # version avant normalisation
adata_raw.layers["counts"] = adata_raw.X.copy()

# Normalisation
adata_norm = adata_raw.copy()  # on crée une copie pour la version normalisée
sc.pp.normalize_total(adata_norm, target_sum=1e3)

# Log
adata_log = adata_norm.copy()
sc.pp.log1p(adata_log)

# Vérifs
print("Avant normalisation (adata_raw) : min =", adata_raw.X.min(), " / max =", adata_raw.X.max())
sums_before = np.ravel(adata_raw.X.sum(axis=1))
print("Sommes avant (ex) :", sums_before[:5])

print("\nAprès normalisation (adata_norm, avant log) : min =", adata_norm.X.min(), " / max =", adata_norm.X.max())
sums_after = np.ravel(adata_norm.X.sum(axis=1))
print("Sommes après (ex) :", sums_after[:5])

print("\nAprès log (adata_log) : min =", adata_log.X.min(), " / max =", adata_log.X.max())

# Attention : pas de normalisation/log si mode=log2

In [ ]:
# 🎨 Visualisation de la normalisation

# Sommes totales par cellule
sums_before = np.ravel(adata_raw.X.sum(axis=1))
sums_after  = np.ravel(adata_norm.X.sum(axis=1))

# On affiche les 100 premières cellules
n_show = min(100, len(sums_before))
x = np.arange(n_show)

plt.figure(figsize=(12,4))

# Avant normalisation
plt.subplot(1,2,1)
plt.bar(x, sums_before[:n_show], color="skyblue", width=0.9)
plt.title("Avant normalisation")
plt.xlabel("Cellules (index)")
plt.ylabel("Total counts par cellule")
plt.ylim(0, max(1050, sums_after.max()*1.05))

# Après normalisation
plt.subplot(1,2,2)
plt.bar(x, sums_after[:n_show], color="lightcoral", width=0.4)
plt.title("Après normalisation (somme ≈ 10 000)")
plt.xlabel("Cellules (index)")
plt.ylim(0, max(1050, sums_after.max()*1.05))

plt.tight_layout()
plt.show()

In [ ]:
# VISUALISATION DE L'EFFET DU LOG

x_raw  = adata_norm.X.flatten()
x_log  = adata_log.X.flatten()

nz_raw = x_raw[x_raw>0]
nz_log = x_log[x_log>0]

fig, axs = plt.subplots(1,2, figsize=(10,4))

axs[0].hist(nz_raw, bins=100, range=(0,200), alpha=0.9)
axs[0].set_title("Avant log (non-zéros)"); axs[0].set_xlabel("Valeurs brutes"); axs[0].set_ylabel("Occurrences")

axs[1].hist(nz_log, bins=100, range=(0,6), alpha=0.9, color="tomato")
axs[1].set_title("Après log1p (non-zéros)"); axs[1].set_xlabel("Valeurs log1p")

plt.tight_layout(); plt.show()

In [ ]:
# 3) HVG

sc.pp.highly_variable_genes(adata_log, n_top_genes=500, flavor="seurat")

adata_hvg = adata_log[:, adata_log.var['highly_variable']].copy()  # après sélection HVG

print(f"Avant HVG : {adata_log.shape}")
print(f"Après HVG : {adata_hvg.shape}")


In [ ]:
sc.pl.highly_variable_genes(adata_log)

In [ ]:
sc.pp.pca(adata_hvg, n_comps=50, svd_solver="randomized", random_state=0)

In [ ]:
sc.pl.pca_variance_ratio(adata_hvg, log=True)

vr = adata_hvg.uns['pca']['variance_ratio']
cum = vr.cumsum()
import matplotlib.pyplot as plt
plt.figure(figsize=(5,4))
plt.plot(range(1, len(vr)+1), vr, marker='o')
plt.xlabel("PC"); plt.ylabel("Variance expliquée")
plt.title("Scree plot")
plt.tight_layout(); plt.show()

plt.figure(figsize=(5,4))
plt.plot(range(1, len(cum)+1), cum, marker='o')
plt.xlabel("PC"); plt.ylabel("Variance cumulée")
plt.title("Variance cumulée")
plt.tight_layout(); plt.show()

In [ ]:
# Graphe des voisins et UMAP

sc.pp.neighbors(adata_hvg, n_neighbors=15, n_pcs=10, random_state=0)


sc.tl.umap(adata_hvg)

In [ ]:
ax2 = ax = sc.pl.umap(adata_hvg, return_fig=False)

In [ ]:
sc.tl.leiden(adata_hvg, resolution=0.008, key_added="leiden")

ax2 = ax = sc.pl.umap(adata_hvg, color="leiden", return_fig=False)

In [ ]:
sc.tl.leiden(adata_hvg, resolution=5, key_added="leiden_pour_fusion")

ax2 = ax = sc.pl.umap(adata_hvg, color="leiden_pour_fusion", return_fig=False)

In [ ]:
### FUSION DE CLUSTERS PAR SCORE

In [ ]:
#1 — Paramètres et listes

# ---- Listes de gènes ----
exc = ["Slc17a7","Slc17a6","Neurod6","Tbr1","Bcl11b","Satb2"]
inh = ["Slc32a1","Gad2","Pvalb","Gad1","Slc6a1"]
non = ["Aqp4","Mog","Cldn5","Rgs5","Sall1","Vcan","Dcn","Pdgfrb","Acta2"]

gene_sets = {"exc": exc, "inh": inh, "non": non}

# ---- Forbidden markers (pour nettoyer chaque macrotype) ----
forbidden = {
    "exc": {
        "inh": ["Gad2","Slc32a1"],
        "non": ["Aqp4","Mog","Cldn5","Sall1","Rgs5","Acta2","Vcan","Dcn"],
    },
    "inh": {
        "exc": ["Slc17a7","Slc17a6","Neurod6","Tbr1","Bcl11b","Satb2"],
        "non": ["Aqp4","Mog","Cldn5","Sall1","Rgs5","Acta2","Vcan","Dcn"],
    },
    "non": {
        "exc": ["Slc17a7","Slc17a6","Neurod6","Tbr1","Bcl11b","Satb2"],
        "inh": ["Gad1","Gad2","Slc32a1","Slc6a1","Pvalb"],
    },
}

# ---- Seuils ----
thr_vs = 0.80                 # seuil diff score (macro vs autres)
thr_cluster_margin = 0.20
thr_forbidden = 0.02          # seuil forbidden moyen (log)

# ---- Nom du clustering ----
groupby = "leiden_pour_fusion"

In [ ]:
#2 — Fonction utilitaire

def mean_expr_X(adata, genes):
    genes = [g for g in genes if g in adata.var_names]
    if len(genes) == 0:
        return np.zeros(adata.n_obs, dtype=float)
    X = adata[:, genes].X
    X = X.toarray() if hasattr(X, "toarray") else X
    return np.asarray(X).mean(axis=1)

In [ ]:
#3 — Scores par cellule

# Scores moyens par cellule pour chaque macrotype
for k, genes in gene_sets.items():
    adata_log.obs[f"score_{k}"] = mean_expr_X(adata_log, genes)

# Différences (macro - autres) + filtres stricts
for k in ["exc","inh","non"]:
    others = [o for o in ["exc","inh","non"] if o != k]
    for o in others:
        adata_log.obs[f"{k}_vs_{o}"] = adata_log.obs[f"score_{k}"] - adata_log.obs[f"score_{o}"]
    
    adata_log.obs[f"{k}_strict_cell"] = np.logical_and.reduce(
        [adata_log.obs[f"{k}_vs_{o}"] > thr_vs for o in others]
    )

In [ ]:
#4 - Report dans adata_hvg

cols = []

# scores
cols += [f"score_{k}" for k in ["exc","inh","non"]]

# diffs + strict
for k in ["exc","inh","non"]:
    for o in ["exc","inh","non"]:
        if o != k:
            cols.append(f"{k}_vs_{o}")
    cols.append(f"{k}_strict_cell")

for c in cols:
    adata_hvg.obs[c] = adata_log.obs[c].values

In [ ]:
#5 - Annotation au niveau cluster

cluster_scores = (
    adata_hvg.obs.groupby(groupby)[["score_exc","score_inh","score_non"]]
    .mean()
)

cluster_scores["macrotype"] = cluster_scores.idxmax(axis=1).str.replace("score_","")

def _top2(row):
    s = row.sort_values(ascending=False)
    return s.iloc[1]

cluster_scores["top1"] = cluster_scores[["score_exc","score_inh","score_non"]].max(axis=1)
cluster_scores["top2"] = cluster_scores[["score_exc","score_inh","score_non"]].apply(_top2, axis=1)
cluster_scores["margin"] = cluster_scores["top1"] - cluster_scores["top2"]
cluster_scores["ambiguous"] = cluster_scores["margin"] < thr_cluster_margin

cluster_scores

In [ ]:
#6) Mapping cluster -> macrotype

mapping = cluster_scores["macrotype"].to_dict()

adata_hvg.obs["macrotype_3"] = adata_hvg.obs[groupby].map(mapping).astype("category")
adata_hvg.obs["cluster_ambiguous"] = (
    adata_hvg.obs[groupby].map(cluster_scores["ambiguous"].to_dict()).astype(bool)
)

In [ ]:
#7) Sélection “pure” pour exc et inh et non

for k in ["exc","inh","non"]:
    k_clusters_conf = cluster_scores.index[
        (cluster_scores["macrotype"] == k) & (~cluster_scores["ambiguous"])
    ].astype(str)

    adata_hvg.obs[f"{k}_from_cluster"] = (
        adata_hvg.obs[groupby].astype(str).isin(k_clusters_conf)
    )

    adata_hvg.obs[f"{k}_pure"] = (
        adata_hvg.obs[f"{k}_from_cluster"] &
        adata_hvg.obs[f"{k}_strict_cell"]
    )

In [ ]:
#8) Nettoyage biologique

# Calcul forbidden sur adata_log + report + filtre final
for k in ["exc","inh","non"]:
    # forbidden markers = marqueurs des "autres" à exclure
    forb_cols = []
    for o in ["exc","inh","non"]:
        if o == k:
            continue
        forb_list = forbidden[k][o]
        col = f"forbidden_{o}_in_{k}"
        adata_log.obs[col] = mean_expr_X(adata_log, forb_list)
        adata_hvg.obs[col] = adata_log.obs[col].values
        forb_cols.append(col)

    adata_hvg.obs[f"{k}_pure_clean"] = (
        adata_hvg.obs[f"{k}_pure"] &
        np.logical_and.reduce([adata_hvg.obs[c] < thr_forbidden for c in forb_cols])
    )

In [ ]:
#9) Résumé et contrôles (pour les 3)

print(cluster_scores.sort_index())

print("\nRépartition macrotype_3:")
print(adata_hvg.obs["macrotype_3"].value_counts())

for k in ["exc","inh","non"]:
    print(f"\n--- {k.upper()} ---")
    print("strict (cellule):", int(adata_hvg.obs[f"{k}_strict_cell"].sum()))
    print("pure (cluster + seuils):", int(adata_hvg.obs[f"{k}_pure"].sum()))
    print("pure clean:", int(adata_hvg.obs[f"{k}_pure_clean"].sum()))

In [ ]:
#10 — UMAP

sc.pl.umap(
    adata_hvg,
    color=[
        groupby, "macrotype_3", "cluster_ambiguous",
        "score_exc", "score_inh", "score_non",
        "exc_vs_inh", "exc_vs_non",
        "inh_vs_exc", "inh_vs_non",
        "non_vs_exc", "non_vs_inh",
        "exc_strict_cell", "exc_pure", "exc_pure_clean",
        "inh_strict_cell", "inh_pure", "inh_pure_clean",
        "non_strict_cell", "non_pure", "non_pure_clean",
        "forbidden_inh_in_exc", "forbidden_non_in_exc",
        "forbidden_exc_in_inh", "forbidden_non_in_inh",
        "forbidden_exc_in_non", "forbidden_inh_in_non",
    ]
)

In [ ]:
# SOUS-TYPE : EXCITATEURS

In [ ]:
# 0) créer adata_exc
adata_exc = adata_log[adata_hvg.obs["exc_pure_clean"]].copy()

# Gènes excitateurs à forcer
force_genes_exc = ["Slc17a7","Slc17a6","Neurod6","Bcl11b","Satb2"]

# 1) HVG
sc.pp.highly_variable_genes(
    adata_exc,
    flavor="seurat",
    n_top_genes=250
)

# 2) Forcer certains gènes à être gardés
force_genes_present_exc = [g for g in force_genes_exc if g in adata_exc.var_names]
print("Gènes forcés présents :", force_genes_present_exc)

keep_genes = (
    adata_exc.var["highly_variable"] |
    adata_exc.var_names.isin(force_genes_present_exc)
)

# 3) Sous-ensemble final
adata_exc = adata_exc[:, keep_genes].copy()

print("Nb gènes après HVG + forçage :", adata_exc.n_vars)

In [ ]:
sc.pp.scale(adata_exc, max_value=10)
sc.tl.pca(adata_exc, n_comps=40)

In [ ]:
sc.pp.neighbors(
    adata_exc,
    n_neighbors=15,
    n_pcs=30,
    metric="cosine"
)

In [ ]:
sc.tl.umap(adata_exc)

In [ ]:
sc.tl.leiden(adata_exc, resolution=0.3)
sc.pl.umap(adata_exc, color=["leiden"])

In [ ]:
print("exc_from_cluster:", adata_hvg.obs["exc_from_cluster"].sum())
print("exc_strict_cell:", adata_hvg.obs["exc_strict_cell"].sum())
print("exc_pure:", adata_hvg.obs["exc_pure"].sum())
print("exc_pure_clean:", adata_hvg.obs["exc_pure_clean"].sum())

In [ ]:
sc.pl.umap(
    adata_exc,
    color=["Fezf2", "Rorb", "Cux2", "Slc17a7"],
)

In [ ]:
# SOUS-TYPE : INHIBITEURS

In [ ]:
# 0) Créer adata_inh
adata_inh = adata_log[adata_hvg.obs["inh_pure_clean"]].copy()

# 1) Gènes inhibiteurs à forcer
force_genes_inh = ["Slc32a1","Gad2","Pvalb","Gad1","Slc6a1", "Lamp5"]

# HVG
sc.pp.highly_variable_genes(
    adata_inh,
    flavor="seurat",
    n_top_genes=250
)

# 2) Forcer certains gènes à être gardés
force_genes_present_inh = [g for g in force_genes_inh if g in adata_inh.var_names]
print("Gènes forcés présents :", force_genes_present_inh)

keep_genes = (
    adata_inh.var["highly_variable"] |
    adata_inh.var_names.isin(force_genes_present_inh)
)

# ) Sous-ensemble final
adata_inh = adata_inh[:, keep_genes].copy()

print("Nb gènes après HVG + forçage :", adata_inh.n_vars)

In [ ]:
sc.pp.scale(adata_inh, max_value=10)
sc.tl.pca(adata_inh, n_comps=40)

In [ ]:
sc.pp.neighbors(
    adata_inh,
    n_neighbors=15,
    n_pcs=30,
    metric="cosine"
)

In [ ]:
sc.tl.umap(adata_inh)

In [ ]:
sc.tl.leiden(adata_inh, resolution=0.3)
sc.pl.umap(adata_inh, color=["leiden"])

In [ ]:
print("inh_from_cluster:", adata_hvg.obs["inh_from_cluster"].sum())
print("inh_strict_cell:", adata_hvg.obs["inh_strict_cell"].sum())
print("inh_pure:", adata_hvg.obs["inh_pure"].sum())
print("inh_pure_clean:", adata_hvg.obs["inh_pure_clean"].sum())

In [ ]:
sc.pl.umap(
    adata_inh,
    color=["Gad2", "Pvalb", "Pax6", "Lamp5", "Calb2"],
)

In [ ]:
# TOP20 gènes exprimés dans le plus grand nombre de cellules (au moins un transcrit)

X = adata_inh.X

# gérer sparse / dense
if hasattr(X, "tocsc"):
    # nombre de cellules où chaque gène est exprimé (>0)
    n_cells = (X > 0).sum(axis=0).A1
else:
    n_cells = (X > 0).sum(axis=0)

# ajouter l’info dans var
adata_inh.var["n_cells_expressing"] = n_cells

# top 20 gènes du plus au moins présent
top20 = (
    adata_inh.var
    .sort_values("n_cells_expressing", ascending=False)
    .head(20)
)

top20

In [ ]:
sc.pl.umap(
    adata_inh,
    color=["Gad2", "Slc32a1", "Nefh", "Serpine2", "Penk", "Fam107a", "Acsbg1", "Dlx6os1", "Pvalb"],
)

In [ ]:
# TOP20 gènes les plus exprimés en moyenne
# Comme tu es en log :
# ce n’est pas une quantité brute,
# mais une intensité moyenne comparable entre gènes

X = adata_inh.X

# moyenne par gène (cells x genes)
if hasattr(X, "mean"):  # sparse ok
    mean_expr = np.asarray(X.mean(axis=0)).ravel()
else:
    mean_expr = X.mean(axis=0)

adata_inh.var["mean_log_expr"] = mean_expr

top20_mean = (
    adata_inh.var
    .sort_values("mean_log_expr", ascending=False)
    .head(20)
)

top20_mean

In [ ]:
sc.pl.umap(
    adata_inh,
    color=["Car8", "Vipr2", "Lgr6", "Adamts19", "Hs3st4", "Glra1", "Klhl14", "Cldn11"],
)

In [ ]:
# TOP20 gènes les plus fortement exprimés en moyenne,
# mais uniquement dans les cellules où ils sont réellement exprimés,
# et cette fois en counts bruts

X = adata_inh.layers["counts"]

# --- nombre de cellules exprimant chaque gène ---
if hasattr(X, "tocsc"):   # sparse
    n_expr = (X > 0).sum(axis=0).A1
    sum_expr = X.sum(axis=0).A1
else:
    n_expr = (X > 0).sum(axis=0)
    sum_expr = X.sum(axis=0)

# --- moyenne conditionnelle (évite la division par 0) ---
mean_counts_expressing = np.zeros_like(sum_expr, dtype=float)
mask = n_expr > 0
mean_counts_expressing[mask] = sum_expr[mask] / n_expr[mask]

# --- ajouter dans .var ---
adata_inh.var["mean_counts_expressing"] = mean_counts_expressing

top20_intensity = (
    adata_inh.var
    .sort_values("mean_counts_expressing", ascending=False)
    .head(20)
)

top20_intensity

In [ ]:
sc.pl.umap(
    adata_inh,
    color=["Pvalb", "Slc32a1", "Penk", "Dlk1", "Calb2", "Ecel1", "Ppp1r1b", "Baiap3"],
)

In [ ]:
# SOUS-TYPE : NON NEURONALES

In [ ]:
# 0) (re)créer adata_non à partir de la matrice complète
adata_non = adata_log[adata_hvg.obs["non_pure_clean"]].copy()

# ---- Liste des gènes non neuronaux à forcer ----
force_genes_non = ["Slc32a1","Gad2","Pvalb","Gad1","Slc6a1"]

# ---- 1) HVG classique ----
sc.pp.highly_variable_genes(
    adata_non,
    flavor="seurat",
    n_top_genes=250
)

# ---- 2) Forcer certains gènes à être gardés ----
force_genes_present_non = [g for g in force_genes_non if g in adata_non.var_names]
print("Gènes forcés présents :", force_genes_present_non)

keep_genes = (
    adata_non.var["highly_variable"] |
    adata_non.var_names.isin(force_genes_present_non)
)

# ---- 3) Sous-ensemble final ----
adata_non = adata_non[:, keep_genes].copy()

print("Nb gènes après HVG + forçage :", adata_non.n_vars)

In [ ]:
sc.pp.scale(adata_non, max_value=10)
sc.tl.pca(adata_non, n_comps=40)

In [ ]:
sc.pp.neighbors(
    adata_non,
    n_neighbors=15,
    n_pcs=30,
    metric="cosine"
)

In [ ]:
sc.tl.umap(adata_non)

In [ ]:
sc.tl.leiden(adata_non, resolution=0.8)
sc.pl.umap(adata_non, color=["leiden"])

In [ ]:
print("non_from_cluster:", adata_hvg.obs["non_from_cluster"].sum())
print("non_strict_cell:", adata_hvg.obs["non_strict_cell"].sum())
print("non_pure:", adata_hvg.obs["non_pure"].sum())
print("non_pure_clean:", adata_hvg.obs["non_pure_clean"].sum())

In [ ]:
sc.pl.umap(
    adata_non,
    color=["Aqp4", "Mog", "Sox10", "Sall1", "Cldn5"],
)